In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
# 경로 설정
train_dir = 'C:/Users/intel/Downloads/trainset(300x300)'
test_dir = 'C:/Users/intel/Downloads/testset(300x300)'

# ImageDataGenerator 설정 
test_datagen = ImageDataGenerator(rescale=1.0/255)
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 배치 단위로 이미지를 불러오기
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),   # 이미지를 300x300으로 리사이즈
    batch_size=32,
    class_mode='sparse',      # 레이블을 정수형으로 반환
    shuffle=True              # 데이터 셔플 (섞기)
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='sparse',
    shuffle=False             # 테스트 데이터는 셔플하지 않음
)

Found 42491 images belonging to 100 classes.
Found 10587 images belonging to 100 classes.


In [5]:
# Pre-trained MobileNet base model without the top dense layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Build a simple model on top of MobileNet
model = Sequential([
    base_model,
    Flatten(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(100, activation='softmax')  # Assume we have 100 classes
])

# Freeze the base model layers if desired
base_model.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

9406464/9406464 [==============================] - 0s 0us/step


In [9]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=test_generator
)

Epoch 1/50
1328/1328 [==============================] - 753s 564ms/step - loss: 4.5883 - accuracy: 0.0199 - val_loss: 4.4795 - val_accuracy: 0.0190
Epoch 2/50
1328/1328 [==============================] - 736s 554ms/step - loss: 4.4885 - accuracy: 0.0189 - val_loss: 4.4736 - val_accuracy: 0.0211
Epoch 3/50
1328/1328 [==============================] - 759s 571ms/step - loss: 4.4850 - accuracy: 0.0194 - val_loss: 4.4533 - val_accuracy: 0.0263
Epoch 4/50
1328/1328 [==============================] - 751s 565ms/step - loss: 4.4787 - accuracy: 0.0200 - val_loss: 4.4474 - val_accuracy: 0.0310
Epoch 5/50
1328/1328 [==============================] - 768s 578ms/step - loss: 4.4715 - accuracy: 0.0220 - val_loss: 4.4351 - val_accuracy: 0.0317
Epoch 6/50
1328/1328 [==============================] - 745s 561ms/step - loss: 4.4651 - accuracy: 0.0232 - val_loss: 4.4175 - val_accuracy: 0.0338
Epoch 7/50
1328/1328 [==============================] - 744s 560ms/step - loss: 4.4583 - accuracy: 0.0245 - val_

KeyboardInterrupt: 

In [ ]:
# 시각화
import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy)+1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.legend()
plt.show()